In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name



TypeError: Runtime.__init__() got an unexpected keyword argument 'boto_session'

In [ ]:

agentcore_runtime = Runtime()

agent_name = "deep_research_scoping_agent"
response = agentcore_runtime.configure(
    entrypoint="agent_core/scoping_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="agent_core/requirements.txt",
    region=region,
    agent_name=agent_name
)
response

In [13]:
launch_result = agentcore_runtime.launch()

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'deep_research_scoping_agent' to account 482387069690 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: deep_research_scoping_agent
ECR repository available: 482387069690.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-deep_research_scoping_agent
Getting or creating execution role for agent: deep_research_scoping_agent
Using AWS region: us-west-2, account ID: 482387069690
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-a59a4561a2


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-deep_research_scoping_agent


✅ Reusing existing execution role: arn:aws:iam::482387069690:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-a59a4561a2
Execution role available: arn:aws:iam::482387069690:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-a59a4561a2
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: deep_research_scoping_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-a59a4561a2
Reusing existing CodeBuild execution role: arn:aws:iam::482387069690:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-a59a4561a2
Using dockerignore.template with 43 patterns for zip filtering
Uploaded source to S3: deep_research_scoping_agent/source.zip
Updated CodeBuild project: bedrock-agentcore-deep_research_scoping_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 9.4s
🔄 DOWNLOAD_SO

In [14]:
payload = {
    "prompt": "I want to research the best coffee shops in San Francisco.",
    "thread_id": "test-thread-1"
}
result = agentcore_runtime.invoke(payload)




RuntimeClientError: An error occurred (RuntimeClientError) when calling the InvokeAgentRuntime operation: An error occurred when starting the runtime. Please check your CloudWatch logs for more information.